In [1]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      1.0.0
aiohttp                      3.8.1
aiohttp-cors                 0.7.0
aioredis                     1.3.1
aiosignal                    1.2.0
ansicon                      1.89.0
argon2-cffi                  21.1.0
astunparse                   1.6.3
async-timeout                4.0.1
attrs                        21.2.0
backcall                     0.2.0
bleach                       4.1.0
blessed                      1.19.0
cachetools                   4.2.4
certifi                      2021.10.8
cffi                         1.15.0
charset-normalizer           2.0.7
click                        8.0.3
colorama                     0.4.4
colorful                     0.5.4
cycler                       0.11.0
debugpy                      1.5.1
decorator                    5.1.0
defusedxml                   0.7.1
Deprecated                   1.2.13
entrypoints                  0.3
filel

You should consider upgrading via the 'E:\lambda\labs\ds-test-2\venv2\Scripts\python.exe -m pip install --upgrade pip' command.



opencensus                   0.8.0
opencensus-context           0.1.2
opencv-python                4.5.4.58
opt-einsum                   3.3.0
packaging                    21.2
pandas                       1.3.4
pandocfilters                1.5.0
parso                        0.8.2
pickleshare                  0.7.5
Pillow                       8.4.0
pip                          21.2.4
prometheus-client            0.12.0
prompt-toolkit               3.0.22
protobuf                     3.19.1
psutil                       5.8.0
py-spy                       0.3.11
pyasn1                       0.4.8
pyasn1-modules               0.2.8
pycparser                    2.21
Pygments                     2.10.0
pyparsing                    2.4.7
pyrsistent                   0.18.0
python-dateutil              2.8.2
pytz                         2021.3
pywin32                      302
pywinpty                     1.1.5
PyYAML                       6.0
pyzmq                        22.3.0
qtconsole    

This notebook was executed on:
- Windows 10
- Ryzen 3900x
- 32 GB ram
- Nvidia 3080

# Research question:
is it possible using the sythetic dataset provided by generate.py and samples of user input provided by story_photo_transformer.py to generate coordinates with less than 8pix mean average error.

---

## Hypothesis
Adding convolution layers to the beginning of the model will improve MAE to below baseline levels

In [2]:
#shared experiment variables
global_batch_size = 96
global_epochs = 1000
global_learning_rate =.00015

# experiment groups
# experiment group Q will test networks that have had their dense layers initialized to constants
group_Q = []
group_Q.append({
    "experiment_label" :"q1",
    "experiment_desc":"constant initialization 1 conv layer, 3 FC Dense of 32 Height",
    "num_conv" :"1",
    "initialization":"constant",
})
group_Q.append({
    "experiment_label" :"q2",
    "experiment_desc":"constant initialization 2 conv layer, 3 FC Dense of 32 Height",
    "num_conv" :"2",
    "initialization":"constant",
})
group_Q.append({
    "experiment_label" :"q3",
    "experiment_desc":"constant initialization 3 conv layer, 3 FC Dense of 32 Height",
    "num_conv" :"3",
    "initialization":"constant",
})
# experiment group R will test networks that have the default random initialization
group_R = []
group_R.append({
    "experiment_label" :"r1",
    "experiment_desc":"default random initialization 1 conv layer, 3 FC Dense of 32 Height",
    "num_conv" :"1",
    "initialization":"default",
})
group_R.append({
    "experiment_label" :"r2",
    "experiment_desc":"default random initialization 2 conv layer, 3 FC Dense of 32 Height",
    "num_conv" :"2",
    "initialization":"default",
})
group_R.append({
    "experiment_label" :"r3",
    "experiment_desc":"default random initialization 3 conv layer, 3 FC Dense of 32 Height",
    "num_conv" :"3",
    "initialization":"default",
})
experiments = group_Q+group_R

In [3]:
# graphing function
def display_history(hist,label):
    # plot history

    loss_p = hist.history['loss']
    val_loss_p = hist.history['val_loss']

    plt.figure(figsize=(15,5))
    plt.plot(loss_p, label='MAE (training data)')
    plt.title(label)
    plt.ylabel('MAE value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper left")
    #plt.yscale('log',base=2)
    plt.show()

    plt.figure(figsize=(15,5))
    plt.plot(val_loss_p, label='MAE (validation data)')
    plt.title(label)
    plt.ylabel('MAE value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper left")
    #plt.yscale('log',base=2)
    plt.show()

    for i in range(0,len(hist.history['val_loss']),1000):
        min_loss = np.array(hist.history['loss'])[:i+1000].min()
        min_val =  np.array(hist.history['val_loss'])[:i+1000].min()
        print(f'By epoch {i+1000} The minimum MAE achieved by the training was {min_loss}.')
        print(f'By epoch {i+1000} The minimum MAE achieved on the real data was {min_val}.')

In [4]:
## section that handles loading data
import os
os.add_dll_directory(r'e:\cuda\bin')

import json
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import cv2
import glob

# this is the path to the synthetic data generated by generate.py, you may need to change this to match your system.
data_dir =r'..\..\..\models\synthetic_data\synthetic_data_for_pipeline_transform\data\*\*'
example_data_files = glob.glob(data_dir)[0:]
print(f'Example of data {example_data_files[0:3]}, {len(example_data_files)} records')

# load each of the X_input files using opencv and convert to BGR colorspace as required by opencv
X_train =[ cv2.cvtColor(cv2.imread(x),cv2.COLOR_RGB2BGR) for x in example_data_files if "X_input" in x ]
# load point data for each record
y_train =[ json.loads( open(y,'rb').read() )["y_label_points"] for y in example_data_files if ".json" in y ]

# convert lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train).reshape((X_train.shape[0],8))

# load real data
# this loads actual recorded user data that optimally we would like to predict with zero error.

path_real = r'..\..\..\data\transcribed_stories\*\*\phase0\*'
real_set=glob.glob(path_real)
# load and convert the data
X_real =np.array([ cv2.cvtColor(cv2.imread(x),cv2.COLOR_RGB2BGR) for x in real_set if "X_input" in x ])
y_real =[ json.loads(open(y,'rb').read() )["y_label_points"] for y in real_set if ".json" in y ]
y_real = np.array(y_real).reshape((X_real.shape[0],8))

assert (X_real.shape[0]==y_real.shape[0])
print(f'Sythetic  data contains {X_train.shape[0]} records.')
print(f'Real data contains {X_real.shape[0]} records.')

Example of data [], 0 records
Sythetic  data contains 0 records.
Real data contains 15 records.


In [5]:
def create_experiment_model(num_conv,initialization):
    if initialization == "constant":
        c_init = keras.initializers.Constant(.004)
        b_init = keras.initializers.Constant(.004)
    else:
        c_init=None
        b_init=None

    img_inputs = keras.Input(shape=(256, 256-64, 3))
    x = img_inputs

    for _ in range(int(num_conv)):
        x = layers.Conv2D(32,(3,3),(2,2))(x)
        x = tf.keras.layers.ReLU()(x)
        x = layers.MaxPool2D(strides=(2,2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(32, activation="relu",dtype="float32",kernel_initializer=c_init,bias_initializer=b_init)(x)
    x = layers.Dense(32, activation="relu",dtype="float32",kernel_initializer=c_init,bias_initializer=b_init)(x)
    x = layers.Dense(32, activation="relu",dtype="float32",kernel_initializer=c_init,bias_initializer=b_init)(x)

    outputs = layers.Dense(8,activation="relu",kernel_initializer=c_init,bias_initializer=b_init)(x)

    model = keras.Model(inputs=img_inputs, outputs=outputs, name="FC_Model")
    model.summary()
    model.compile(loss="MAE",optimizer=tf.keras.optimizers.Adam(learning_rate=global_learning_rate))
    return model

def run_experiment(model):
    with tf.device("GPU"):
        history = model.fit(X_train,y_train,epochs=global_epochs,validation_data=(X_real,y_real),shuffle=False,verbose=0,
                            batch_size=global_batch_size)

        return history

In [6]:
start_time = time.time()
results={}
for E in experiments:
    print("\n")
    print( f'=== Experiment {E["experiment_label"]} ===')
    print( E["experiment_desc"])
    model = create_experiment_model(E["num_conv"],E["initialization"])
    history = run_experiment(model)
    display_history(history,E["experiment_desc"])

    results[E["experiment_label"]]={
    "min_val_mae":np.array(history.history["val_loss"]).min()
    }
print (f'number of seconds to run all experiments: {time.time()-start_time}')



=== Experiment q1 ===
constant initialization 1 conv layer, 3 FC Dense of 32 Height
Model: "FC_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 192, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 127, 95, 32)       896       
                                                                 
 re_lu (ReLU)                (None, 127, 95, 32)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 47, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 94752)             0         
                                                                 
 dense (Dense)               (None, 32

ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.